In [1]:
###### FOR PACE ICE - replace GT username below ######
%cd /storage/ice1/1/5/rpatel627/datasets/coco/vlm_attention-main
!pwd

/storage/ice1/1/5/rpatel627/datasets/coco/vlm_attention-main
/storage/ice1/1/5/rpatel627/datasets/coco/vlm_attention-main


In [2]:
# !pip install -q -U google-generativeai
# !pip install -q transformers accelerate bitsandbytes

In [3]:
import os
import json
import base64
from openai import OpenAI
import google.generativeai as genai
from dotenv import load_dotenv
from tqdm import tqdm

In [ ]:
# load_dotenv()
# API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_API_KEY = ""
GOOGLE_API_KEY = ""


In [9]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)
genai.configure(api_key=GOOGLE_API_KEY)


In [10]:
if not OPENAI_API_KEY:
    raise ValueError(" OPENAI_API_KEY not found in .env file")
if not GOOGLE_API_KEY:
    raise ValueError("Google API Key is missing")
    
print("Available models for your key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)
    
print("Available models for your key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Available models for your key:
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-fla

In [11]:
IMAGE_FOLDER = "images"           
#OUTPUT_PATH = "blip_results/captions.jsonl"
OUTPUT_PATH = "captions.jsonl"
GPT_MODEL = "gpt-4.1-mini"
GOOGLE_MODEL = genai.GenerativeModel('models/gemini-2.0-flash')
MAX_OUTPUT = 200               

In [12]:
def encode_image(image_path):
    with open(image_path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

In [13]:
def generate_questions(base64_image):
    prompt = """
You are preparing controlled experimental materials for multimodal evaluation.

Given the IMAGE (provided separately), generate the following:

----------------------------------------------------------------------
1. A correct caption
----------------------------------------------------------------------
• Must accurately describe the visible scene.
• 7–15 words, objective, simple, and factual.
• Must mention the main subject(s) and one key attribute
  (e.g., species, color, object type, action, or spatial relation).
• Should be worded well and clearly. 

----------------------------------------------------------------------
2. A deliberately incorrect caption
----------------------------------------------------------------------
• Must keep the same length and sentence structure style as the correct caption.
• MUST change EXACTLY TWO meaningful visual attributes from the correct caption.
  Allowed attribute types:
     – species/category of the main object
     – color of a main object
     – pattern/texture of a main object
     – object type that a person is holding/using
     – action the main subject is performing
     – spatial relation (e.g., “in front of” → “behind”)
     
• The incorrect caption MUST be **factually wrong for THIS image**.
  It should contradict TWO concrete visual facts visible in the picture, not merely
  describe an alternative plausible real-world scenario -- VERY IMPORTANT!
  (Example: If the scene shows a lake, “ocean” is *not* allowed because both can
   coexist conceptually; the changed attributes must be unambiguously false.)

• The incorrect caption must remain syntactically valid and plausible for the 
  kind of world the image depicts, but factually wrong.

• The two changed attributes MUST be *the most visually important attributes*
  from the correct caption.

----------------------------------------------------------------------
3. Five Visual Necessity Ladder (VNL) questions (L0–L4)
----------------------------------------------------------------------

L0 — Pure language prior  
• Must be answerable with NO access to the image.  
• General world knowledge only; do NOT reference animals, people,
  objects, nature, or environments.  
• 6–14 words.

L1 — Probe changed attribute #1 
• MUST directly probe the FIRST changed attribute from the incorrect caption.  
• Example:If species changed, ask “What type of animal…?”  
          If color changed, ask “What color is…?”  
          If object type changed, ask “What object is… holding?”  
• No attributes other than the first changed one.  
• 6–14 words.

L2 — Probe changed attribute #2
• MUST directly probe the SECOND changed attribute from the incorrect caption.  
• Same rules as L2 but targeting the second changed detail.  
• Should not be the same question as L1. 
• 6–14 words.

L3 — High-level reasoning
• Ask a reasoning question that is loosely related to the scene shown in the image.
• The question MUST NOT depend on the two changed attributes.
• The question MUST NOT target the same object/attribute as L1 or L2.
• The question SHOULD require general common-sense or contextual reasoning.
• The question SHOULD still be answerable using the image (but only its general context, not the altered details).
• 6–14 words.

----------------------------------------------------------------------
GENERAL RULES
----------------------------------------------------------------------
• Do NOT provide answers.
• Do NOT describe the image outside captions.
• All questions must be 6–14 words.
• Output MUST be a single JSON object in the exact format below.

----------------------------------------------------------------------
Return EXACTLY this JSON structure:
----------------------------------------------------------------------
{
  "correct_caption": "<string>",
  "incorrect_caption": "<string>",
  "L0": "<string>",
  "L1": "<string>",    // targets changed attribute #1
  "L2": "<string>",    // targets changed attribute #2
  "L3": "<string>"   
}


"""
    response = openai_client.responses.create(
        model=GPT_MODEL,
        max_output_tokens=MAX_OUTPUT,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {
                        "type": "input_image",
                        "image_url": f"data:image/jpeg;base64,{base64_image}"
                    }
                ]
            }
        ]
    )

    return json.loads(response.output_text)

In [16]:
###### FOR PACE ICE ONLY - replace GT username below ######

# Tells HuggingFace to save all downloaded models + datasets in scratch directory instead of home directory
os.environ["HF_HOME"] = "/home/hice1/rpatel627/scratch/huggingface"
os.environ["HF_DATASETS_CACHE"] = "/home/hice1/rpatel627/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/home/hice1/rpatel627/scratch/hf_cache"

In [17]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch
from PIL import Image

/home/hice1/rpatel627/scratch/.conda/envs/finalProj/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [20]:
from transformers import AutoProcessor, LlavaForConditionalGeneration, AutoConfig
import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Using device:", device)

MODEL_NAME = "llava-hf/llava-1.5-7b-hf"
config = AutoConfig.from_pretrained(MODEL_NAME)
config.output_attentions = True
config.return_dict_in_generate = True

print("Loading LLaVA model with attentions enabled...")
processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = LlavaForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    config=config,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device)

print("✅ LLaVA Model loaded!")

✅ Using device: cuda
Loading LLaVA model with attentions enabled...


Some kwargs in processor config are unused and will not have any effect: num_additional_image_tokens. 


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ LLaVA Model loaded!


In [ ]:
def compute_llava_mdi(attentions, inputs, image_token_id=32000, vision_token_count=576):
    """
    Computes MDI by detecting image token position and accounting for LLaVA's 
    internal token expansion (1 token -> 576 embeddings).
    """
    if not attentions or len(attentions) == 0:
        return None
    
    # 1. Find where the <image> token is in the input_ids
    input_ids = inputs.input_ids[0]  # [seq_len]
    image_indices = torch.where(input_ids == image_token_id)[0]
    
    if len(image_indices) == 0:
        return None
    
    # Position of <image> token in input_ids
    img_token_pos = image_indices[0].item()
    
    # 2. Calculate actual position in attention matrix
    # In input_ids: [token_0, token_1, ..., token_img_token_pos (=<image>), ..., token_n]
    # In attention:  [token_0, token_1, ..., [576 vision embeddings], ..., token_n]    
    vis_start = img_token_pos
    vis_end = img_token_pos + vision_token_count

    visual_scores = []
    textual_scores = []
    
    # Iterate over generated tokens (outer tuple) and layers (inner tuple)
    for token_step_attentions in attentions:
        for layer_attention in token_step_attentions:
            
            # layer_attention shape: [batch, heads, query_len, key_len]
            avg_attention = layer_attention.mean(dim=(0, 1)) 
            
            # Attention of the newly generated token looking back at context
            final_attn_row = avg_attention[-1, :]
            
            total_len = final_attn_row.shape[0]
            
            # Debug
            # print(f"Image starts at {vis_start}, ends at {vis_end}, Total Seq Len: {total_len}")
            
            if vis_end <= total_len:
                # Attention on visual tokens
                visual_val = final_attn_row[vis_start:vis_end].sum().item()
                
                # Attention on text tokens (before and after image)
                text_before = final_attn_row[:vis_start].sum().item()
                text_after = final_attn_row[vis_end:].sum().item()
                
                textual_val = text_before + text_after
            else:
                # Fallback if dimensions don't match
                visual_val = 0.0
                textual_val = final_attn_row.sum().item()

            print(f"Total attention length: {total_len}, "
                  f"Image tokens: [{vis_start}:{vis_end}], "
                  f"Visual attention: {visual_val:.4f}, "
                  f"Text attention: {textual_val:.4f}")
            visual_scores.append(visual_val)
            textual_scores.append(textual_val)
    
    if not visual_scores:
        return None
    
    # Average over all layers and generated tokens
    avg_vis = sum(visual_scores) / len(visual_scores)
    avg_text = sum(textual_scores) / len(textual_scores)
    
    # Compute MDI
    mdi = avg_vis / (avg_vis + avg_text + 1e-9)
    return mdi

In [ ]:
def ask_llava(
    image_path, 
    caption, 
    question, 
    history, 
    max_new_tokens=50, 
    return_metrics=False,
    last_turn_only=False
):
    image = Image.open(image_path).convert("RGB")

    # 1. Base Prompt
    base_prompt = f"USER: <image>\nContext: {caption}\n"
    
    # 2. History
    conversation_so_far = history 
    if last_turn_only and history:
         # Note: History in your script was a string, adapt if it's a list
         pass 

    # 3. Final Prompt
    final_prompt = f"{base_prompt}{conversation_so_far}USER: {question}\nASSISTANT:"

    # 4. Process Inputs
    inputs = processor(
        text=final_prompt, 
        images=image, 
        return_tensors="pt"
    ).to(model.device, dtype=torch.float16)

    # 5. Generate with Attentions
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            output_attentions=True,       
            return_dict_in_generate=True  
        )

    # 6. Decode Answer
    generated_ids = outputs.sequences[0][inputs.input_ids.shape[1]:] # Slice off input
    answer = processor.decode(generated_ids, skip_special_tokens=True).strip()

    # 7. Compute Metrics
    if return_metrics:
        # Compute MDI - pass the entire attentions tuple and inputs
        final_mdi = compute_llava_mdi(outputs.attentions, inputs) 
        if final_mdi is None:
            final_mdi = 0.0

        # Return tuple
        return answer, final_mdi

    return answer

In [43]:
def create_eval_prompt(caption, condition, question, model_answer):
    return f"""
You are an automated scoring module. You MUST output ONLY a single character: "0" or "1".
NEVER output words, sentences, explanations, punctuation, or reasoning.

YOUR TASK:
Judge whether the MODEL_ANSWER correctly matches what is visually true in the IMAGE.

IMPORTANT:
• The CAPTION was only input context for another model. Do NOT treat it as truth.
• Ignore any misleading or incorrect information in the CAPTION.
• Use ONLY the IMAGE + the QUESTION + world knowledge to judge correctness.
• If the MODEL_ANSWER matches the visible truth in the image, output "1".
• If the MODEL_ANSWER is wrong, unsupported, or contradicts the image, output "0".
• Your ENTIRE reply MUST be exactly one character: "0" or "1".

Now evaluate:

CAPTION (context only): {caption}
CONDITION: {condition}
QUESTION: {question}
MODEL_ANSWER: {model_answer}

Reply with ONLY "0" or "1".
"""

In [44]:
def eval_answer(image_path, caption, condition, question, model_answer):
    """
    Uses Google Gemini to judge the answer.
    Returns 0 or 1.
    """
    
    # 1. Load the image using PIL (Gemini likes this format)
    img = Image.open(image_path)
    
    # 2. Get your prompt text
    # (Keep your create_eval_prompt function exactly the same as it is now)
    prompt_text = create_eval_prompt(caption, condition, question, model_answer)
    
    try:
        # 3. Call Gemini
        # We pass the text prompt AND the image object in a list
        response = GOOGLE_MODEL.generate_content([prompt_text, img])
        
        # 4. Clean the output
        output = response.text.strip()
        
        # Gemini might be chatty, ensure we just get the number
        if "1" in output: return 1
        if "0" in output: return 0
        
        # Fallback if it returns something else
        return 0
        
    except Exception as e:
        print(f"Gemini Error: {e}")
        return 0

In [45]:
def compute_fooling_by_level(jsonl_path):
    stats = {
        "L0": {"fooled": 0, "total": 0},
        "L1": {"fooled": 0, "total": 0},
        "L2": {"fooled": 0, "total": 0},
        "L3": {"fooled": 0, "total": 0}
    }

    with open(jsonl_path, "r") as f:
        for line in f:
            item = json.loads(line)
            eval_scores = item["eval_scores"]

            # ensure consistent level order
            for level in ["L0", "L1", "L2", "L3"]:
                scores = eval_scores[level]

                s_c = scores["correct_caption_score"]
                s_i = scores["incorrect_caption_score"]

                stats[level]["total"] += 1

                # Fooling event = correct under correct caption AND incorrect under incorrect caption
                if s_c == 1 and s_i == 0:
                    stats[level]["fooled"] += 1

    # compute fooling rates
    for lvl in stats:
        total = stats[lvl]["total"]
        fooled = stats[lvl]["fooled"]
        stats[lvl]["rate"] = fooled / total if total > 0 else 0.0

    return stats

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def run():
    image_files = [
        f for f in os.listdir(IMAGE_FOLDER)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"Found {len(image_files)} images.\n")

    with open(OUTPUT_PATH, "w", encoding="utf-8") as out:
        for img_file in tqdm(image_files, desc="Processing"):
            image_id = os.path.splitext(img_file)[0]
            path = os.path.join(IMAGE_FOLDER, img_file)

            try:
                # ---- 1) GPT captions + questions ----
                b64 = encode_image(path)
                q = generate_questions(b64)

                correct_caption = q["correct_caption"]
                incorrect_caption = q["incorrect_caption"]

                L0 = q["L0"]
                L1 = q["L1"]
                L2 = q["L2"]
                L3 = q["L3"]

                # ---- 2) LLaVA answers + metrics ----
                # --- A. Correct Caption Condition ---
                answers_correct = {}
                mdi_correct = {}

                
                history_correct = ""  
                
                # Iterate through levels
                for lvl, question in [("L0", L0), ("L1", L1), ("L2", L2), ("L3", L3)]:
                    ans, mdi = ask_llava(
                        path, 
                        correct_caption, 
                        question, 
                        history_correct, 
                        return_metrics=True
                    )
                    
                    # Store results
                    answers_correct[lvl] = ans
                    mdi_correct[lvl] = round(mdi, 3) if mdi is not None else None

                    history_correct += f"User: {question}\nAssistant: {ans}\n"

                # --- B. Incorrect Caption Condition ---
                answers_incorrect = {}
                mdi_incorrect = {}
                
                history_incorrect = ""
                
                for lvl, question in [("L0", L0), ("L1", L1), ("L2", L2), ("L3", L3)]:
                    ans, mdi = ask_llava(
                        path,
                        incorrect_caption,
                        question,
                        history_incorrect,
                        return_metrics=True
                    )
                    
                    answers_incorrect[lvl] = ans
                    mdi_incorrect[lvl] = round(mdi, 3) if mdi is not None else None
                    
                    history_incorrect += f"User: {question}\nAssistant: {ans}\n"

                # ---- 3) Base JSON structure (Updated with Metrics) ----
                output = {
                    "image_id": image_id,

                    "captions": {
                        "correct": correct_caption,
                        "incorrect": incorrect_caption
                    },

                    "questions": {
                        "L0": L0,
                        "L1": L1,
                        "L2": L2,
                        "L3": L3
                    },

                    "answers": {
                        "correct_caption": answers_correct,
                        "incorrect_caption": answers_incorrect
                    },
                    
                    # NEW FIELDS FOR METRICS
                    "mdi_scores": {
                        "correct_caption": mdi_correct,
                        "incorrect_caption": mdi_incorrect
                    },

                    "eval_scores": {}   # will be filled next
                }


                # ---- 4) Parallel Claude evaluation ----
                jobs = []
                with ThreadPoolExecutor(max_workers=8) as ex:
                    for level, question in output["questions"].items():

                        # correct caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["correct"],
                            "correct caption condition",
                            question,
                            output["answers"]["correct_caption"][level]
                        ))

                        # incorrect caption condition
                        jobs.append(ex.submit(
                            eval_answer,
                            path,
                            output["captions"]["incorrect"],
                            "incorrect caption condition",
                            question,
                            output["answers"]["incorrect_caption"][level]
                        ))

                    # collect results
                    ordered_results = [j.result() for j in jobs]

                # ---- 5) Attach scores to JSON in correct structure ----
                idx = 0
                for level in ["L0", "L1", "L2", "L3"]:
                    score_c = ordered_results[idx]; idx += 1
                    score_i = ordered_results[idx]; idx += 1

                    output["eval_scores"][level] = {
                        "correct_caption_score": score_c,
                        "incorrect_caption_score": score_i
                    }

                # ---- 6) Write one JSON line ----
                out.write(json.dumps(output, ensure_ascii=False) + "\n")

            except Exception as e:
                print(f"\nError with {image_id}: {e}")
                

    print(f"\nDone. JSONL saved to: {OUTPUT_PATH}\n")
    stats = compute_fooling_by_level(OUTPUT_PATH)
    print(stats)


In [47]:
if __name__ == "__main__":
    run()

Found 1 images.



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.6465, Text attention: 0.3537
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.3591, Text attention: 0.6409
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.1584, Text attention: 0.8413
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.0722, Text attention: 0.9281
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.0620, Text attention: 0.9380
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image tokens: [5:581], Visual attention: 0.0533, Text attention: 0.9468
Image starts at 5, ends at 581, Total Seq Len: 615
Total attention length: 615, Image to

Processing: 100%|██████████| 1/1 [00:10<00:00, 10.91s/it]


Done. JSONL saved to: captions.jsonl

{'L0': {'fooled': 0, 'total': 1, 'rate': 0.0}, 'L1': {'fooled': 0, 'total': 1, 'rate': 0.0}, 'L2': {'fooled': 0, 'total': 1, 'rate': 0.0}, 'L3': {'fooled': 0, 'total': 1, 'rate': 0.0}}
